In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from itertools import product
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from copy import deepcopy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
SABR_cal = pd.read_csv('/kaggle/input/sabr-cal-30/SABR_model_results.csv')
SABR_cal.head()

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,...,sigma_351,sigma_352,sigma_353,S0,alpha,beta,rho,nu,r,T
0,90.422466,90.805575,91.447185,91.613698,92.952651,94.127607,94.297985,93.500430,95.717052,97.809856,...,0.245079,0.243950,0.244192,88.448489,0.242849,1.0,0.203859,0.063728,0.015766,1.4
1,90.422466,91.833751,92.266471,90.639588,89.925484,89.338754,91.262856,90.850083,89.777434,89.554650,...,0.261715,0.261511,0.261084,88.448489,0.242849,1.0,0.203859,0.063728,0.015766,1.4
2,90.422466,90.647801,89.949748,89.232632,87.787124,88.314657,88.183894,86.932109,86.299791,83.874874,...,0.238487,0.237984,0.237329,88.448489,0.242849,1.0,0.203859,0.063728,0.015766,1.4
3,90.422466,90.013999,90.229666,93.225704,94.396989,94.202796,92.748704,92.821197,92.352620,92.208894,...,0.221166,0.222519,0.224034,88.448489,0.242849,1.0,0.203859,0.063728,0.015766,1.4
4,90.422466,92.345150,91.183986,88.825354,90.778707,91.732969,93.124575,92.806635,90.812415,87.677114,...,0.252428,0.250448,0.251429,88.448489,0.242849,1.0,0.203859,0.063728,0.015766,1.4


In [4]:
class OptionPricingDatasetSABR(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32).to(device)
        self.y = torch.tensor(y.values, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
X_sabr_cal = SABR_cal.drop(['alpha','beta','rho','nu'], axis=1)  # Features
y_sabr_cal = SABR_cal[['alpha','rho','nu']]          # Target

X_train_sabr_cal , X_temp_sabr_cal , y_train_sabr_cal , y_temp_sabr_cal  = train_test_split(X_sabr_cal , y_sabr_cal , test_size=0.2, random_state=42)         # Train/Test+Val
X_val_sabr_cal , X_test_sabr_cal , y_val_sabr_cal , y_test_sabr_cal  = train_test_split(X_temp_sabr_cal, y_temp_sabr_cal, test_size=0.5, random_state=42)  # Val/Test


scaler = MinMaxScaler()
X_train_normalized_sabr_cal  = scaler.fit_transform(X_train_sabr_cal)  # Fit on training data and transform
X_val_normalized_sabr_cal  = scaler.transform(X_val_sabr_cal)         # Transform validation data
X_test_normalized_sabr_cal  = scaler.transform(X_test_sabr_cal)       # Transform test data

# Convert normalized data back into DataFrames for DataLoader compatibility
X_train_normalized_sabr_cal  = pd.DataFrame(X_train_normalized_sabr_cal , columns=X_sabr_cal.columns)
X_val_normalized_sabr_cal  = pd.DataFrame(X_val_normalized_sabr_cal , columns=X_sabr_cal.columns)
X_test_normalized_sabr_cal  = pd.DataFrame(X_test_normalized_sabr_cal , columns=X_sabr_cal.columns)

# Create PyTorch datasets
train_dataset_sabr_cal  = OptionPricingDatasetSABR(X_train_normalized_sabr_cal , y_train_sabr_cal)
val_dataset_sabr_cal  = OptionPricingDatasetSABR(X_val_normalized_sabr_cal , y_val_sabr_cal)
test_dataset_sabr_cal  = OptionPricingDatasetSABR(X_test_normalized_sabr_cal , y_test_sabr_cal)

# Create DataLoaders
train_loader_sabr_cal  = torch.utils.data.DataLoader(train_dataset_sabr_cal , batch_size=1024, shuffle=True)
val_loader_sabr_cal  = torch.utils.data.DataLoader(val_dataset_sabr_cal , batch_size=1024, shuffle=False)
test_loader_sabr_cal  = torch.utils.data.DataLoader(test_dataset_sabr_cal , batch_size=1024, shuffle=False)


In [13]:
def train_model_sabr_cal(model, train_loader, val_loader, mse_criterion, optimizer, device, num_epochs=50, patience=5):
    best_loss = float('inf')
    counter = 0
    best_model = deepcopy(model.state_dict())
    all_predictions = []
    all_targets = []
    for epoch in range(num_epochs):
        model.train()
        running_loss_mse = 0.0

        for inputs, targets in tqdm(train_loader, desc="Training Epoch", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            targets = targets.squeeze(1)

            # Compute MSE loss for monitoring (not used in training)
            loss_mse = mse_criterion(outputs, targets)

            # Backward pass and optimization
            loss_mse.backward()
            optimizer.step()

            running_loss_mse += loss_mse.item()
            
        avg_train_loss_mse = running_loss_mse / len(train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs} - "
              f"Train MSE: {avg_train_loss_mse:.10f}")

        #print(f"Epoch {epoch + 1}/{num_epochs},  Train MSE Loss: {avg_train_loss_mse:.10f}")

        # Validation after each epoch
        avg_val_loss_mse = validation_model_sabr_cal(model, val_loader, mse_criterion, device)


        # Early stopping based on custom loss
        if avg_val_loss_mse < best_loss:
            best_loss = avg_val_loss_mse
            counter = 0
            best_model = deepcopy(model.state_dict())  # Save the best model state
        else:
            counter += 1

        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    model.load_state_dict(best_model)  # Restore the best model
    return model, avg_train_loss_mse, avg_val_loss_mse, best_loss

# Validation function
def validation_model_sabr_cal(model, val_loader,  mse_criterion, device):
    model.eval()
    val_loss_mse = 0.0
    #all_predictions = []
    #all_targets = []
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc="Validation", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            targets = targets.squeeze(1)

            loss_mse = mse_criterion(outputs, targets)        # MSE loss

            val_loss_mse += loss_mse.item()

            # Collect predictions and true values for MAE & R² computation
            #all_predictions.extend(outputs.cpu().detach().numpy().flatten())
            #all_targets.extend(targets.cpu().detach().numpy().flatten())
            
        avg_val_loss_mse = val_loss_mse / len(val_loader)
        #avg_val_loss_mae = mean_absolute_error(all_targets, all_predictions)
        #val_r2_score = r2_score(all_targets, all_predictions)

        print(f"Val MSE: {avg_val_loss_mse:.10f}")#, "
              #f"Val  MAE: {avg_val_loss_mae:.10f}, "
              #f"Val  R²: {val_r2_score:.10f}")
    #print(f" Validation MSE Loss: {avg_val_loss_mse:.10f}")
    return  avg_val_loss_mse

# Testing function
def test_model_sabr_cal(model, test_loader, mse_criterion, device):
    model.eval()
    
    test_loss_mse = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="Testing", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            targets = targets.squeeze(1)
            
            loss_mse = mse_criterion(outputs, targets)        # MSE loss

            test_loss_mse += loss_mse.item()

            all_predictions.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    avg_test_loss_mse= mean_squared_error(all_targets,all_predictions)
    avg_test_loss_mae = mean_absolute_error(all_targets, all_predictions)
    test_r2_score = r2_score(all_targets, all_predictions)

    print(f"Test MSE: {avg_test_loss_mse:.10f}, "
          f"Test MAE: {avg_test_loss_mae:.10f}, "
          f"Test R²: {test_r2_score:.10f}")

    return  avg_test_loss_mse, all_predictions, all_targets

In [ ]:
def objective(trial):
    # Define hyperparameters
    num_layers = trial.suggest_int("num_layers", 2, 5)  # Number of hidden layers
    hidden_size = trial.suggest_int("hidden_size", 500, 2000)  # Neurons per layer
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.2)  # Dropout rate
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3, log=True)  # Learning rate
    batch_size = trial.suggest_categorical("batch_size", [256,512,1024,2048])  # Batch size

    # Define the MLP model dynamically
    class MLP(nn.Module):
        def __init__(self, input_dim, num_layers, hidden_size, dropout_rate):
            super(MLP, self).__init__()
            layers = [nn.Linear(input_dim, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)]
            for _ in range(num_layers - 1):
                layers.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.extend([nn.Linear(hidden_size, int(hidden_size/4)), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.extend([nn.Linear(int(hidden_size/4), int(hidden_size/4)), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.append(nn.Linear(int(hidden_size/4), 3))  # Output layer
            self.model = nn.Sequential(*layers)

        def forward(self, x):
            return self.model(x)

    # Initialize model
    input_dim = X_train_sabr_cal.shape[1]
    model = MLP(input_dim, num_layers, hidden_size, dropout_rate).to(device)

    # Define loss function and optimizer
    mse_criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoaders with the suggested batch_size
    train_loader_sabr_cal = torch.utils.data.DataLoader(train_dataset_sabr_cal, batch_size=batch_size, shuffle=True)
    val_loader_sabr_cal = torch.utils.data.DataLoader(val_dataset_sabr_cal, batch_size=batch_size, shuffle=False)

    # Train the model
    trained_model, _, _, best_val_loss = train_model_sabr_cal(
        model, train_loader_sabr_cal, val_loader_sabr_cal, mse_criterion, optimizer, device, num_epochs=50, patience=5
    )
    torch.save(trained_model.state_dict(), "mlp_SABR_cal.pth")
    print("Model saved!")
    # Return validation loss (Optuna minimizes this)
    return best_val_loss  

# Use an SQLite database to store trials
storage = "sqlite:///optuna_study.db"

# Run Optuna study with TPESampler and persistent storage
study = optuna.create_study(
    study_name="MLP_Optimization_vol", 
    direction="minimize", 
    sampler=optuna.samplers.TPESampler(), 
    storage=storage, 
    load_if_exists=True  # Resume if the study exists
)

study.optimize(objective, n_trials=20)  # Run 30 trials

# Print best hyperparameters
print("Best Hyperparameters:", study.best_params)

[I 2025-03-26 17:19:54,712] Using an existing study with name 'MLP_Optimization_vol' instead of creating a new one.
Training Epoch: 100%|██████████| 782/782 [02:16<00:00,  5.71batch/s]


Epoch 1/50 - Train MSE: 0.0606068775


Validation: 100%|██████████| 98/98 [00:05<00:00, 18.61batch/s]


Val MSE: 0.0332750755


Training Epoch: 100%|██████████| 782/782 [02:27<00:00,  5.31batch/s]


...

Training Epoch: 100%|██████████| 3125/3125 [10:54<00:00,  4.77batch/s]


Epoch 3/50 - Train MSE: 0.0093095863


Validation: 100%|██████████| 391/391 [00:15<00:00, 24.95batch/s]


Val MSE: 0.0036225612


Training Epoch:  90%|█████████ | 2823/3125 [09:52<01:01,  4.89batch/s]

In [ ]:
# Test phase
test_loss, predictions, true_values = test_model_sabr_cal(trained_model, test_loader_sabr_cal, mse_criterion, device)

# Save model
torch.save(trained_model.state_dict(), "mlp_SABR_cal_rho.pth")
print("Model saved!")

In [11]:
# Test phase
test_loss, predictions, true_values = test_model_sabr_cal(trained_model, test_loader_sabr_cal, mse_criterion, device)


NameError: name 'trained_model' is not defined

In [37]:
#print(test_loss)
for i in range(len(predictions)):
    if i==30:
        break
    print(f"Prediction: {predictions[i]}, True Value: {true_values[i]}")

NameError: name 'predictions' is not defined

In [2]:
db_path = "/kaggle/input/dboptuna/optuna_study (2).db"  # Replace with your actual path
study_name = "MLP_Optimization_vol"  # Your study name

study = optuna.load_study(study_name=study_name, storage=f"sqlite:///{db_path}")

# Print the best trial
print("Best Trial:")
print(f"  Number: {study.best_trial.number}")
print(f"  Value: {study.best_trial.value}")
print(f"  Params: {study.best_trial.params}")

# Print all trials (optional)
for trial in study.trials:
    print(f"Trial {trial.number}: Value={trial.value}, Params={trial.params}")

Best Trial:
  Number: 7
  Value: 0.0002476581408997418
  Params: {'num_layers': 5, 'hidden_size': 770, 'dropout_rate': 0.058418252474786825, 'learning_rate': 0.00017537517229441845, 'batch_size': 1024}
Trial 0: Value=None, Params={'num_layers': 4, 'hidden_size': 1309, 'dropout_rate': 0.18553568495516168, 'learning_rate': 0.00030904963939730847, 'batch_size': 256}
Trial 1: Value=None, Params={'num_layers': 4, 'hidden_size': 1272, 'dropout_rate': 0.05510190161935529, 'learning_rate': 0.0002711541592576603, 'batch_size': 2048}
Trial 2: Value=None, Params={'num_layers': 2, 'hidden_size': 1042, 'dropout_rate': 0.13704946756440722, 'learning_rate': 0.0001962824540572799, 'batch_size': 512}
Trial 3: Value=0.0058866909191924695, Params={'num_layers': 2, 'hidden_size': 1021, 'dropout_rate': 0.19712927077787873, 'learning_rate': 0.0005317541785270053, 'batch_size': 1024}
Trial 4: Value=0.0011420252040246516, Params={'num_layers': 4, 'hidden_size': 1431, 'dropout_rate': 0.14874841232122568, 'lear